# Resource project scheduling problems

In this notebook, we explore how to solve a resource constrained project scheduling problem (RCPSP).

The problem is made of $M$ activities that have precedence constraints. That means that if activity $j\in[1,M]$ is a successor of activity $i\in[1,M]$, then activity $i$ must be completed before activity $j$ can be started

On top of these constraints, each project is assigned a set of K renewable resources where each resource $k$ is available in $R_{k}$ units for the entire duration of the project. Each activity may require one or more of these resources to be completed. While scheduling the activities, the daily resource usage for resource $k$ can not exceed $R_{k}$ units.

Each activity $j$ takes $d_{j}$ time units to complete.

The overall goal of the problem is usually to minimize the makespan.

A classic variant of RCPSP is the multimode RCPSP where each task can be executed in several ways (one way=one mode). A typical example is :

Mode n°1 'Fast mode': high resource consumption and fast
Mode n°2 'Slow mode' : low resource consumption but slow

## A simple problem definition

Let start with a very simple problem which has only 5 tasks to execute:
All tasks have specific durations and they can consume 1 type of renewable resources.

Some tasks need to be executed after some have completed, for example:
 - Task 1 has 2 successors: task 2 and task 4
 - Task 2 has a single successor: Task 3
 - Task 3 and Task 4 have the same successor: Task 5
 - Task 5 has no successor
 
Task 0 & Task 5 have specific roles. The first one is called the source and the former the sink. They ususally have a zero duration.

In our problem, Tasks 2 & 3 have a duration of 3 while Task 4 has a duration of 7 units.

![image.png](images/sched.png)

In [ ]:
# To load minizinc path etc.
import skdecide.hub

In [ ]:
import sys, os
this_folder = os.getcwd()
sys.path.append(os.path.join(this_folder, "discrete_optimisation/"))

# Missing installation

In [ ]:
!pip install numba
!pip install sortedcontainers
!pip install shapely

## Loading the problems definition

In [ ]:
from discrete_optimization.rcpsp.rcpsp_model import RCPSPModel, RCPSPSolution
from discrete_optimization.rcpsp.rcpsp_parser import files_available, parse_file

In [ ]:
print([os.path.basename(f) for f in files_available])

Now we can load some RCPSP problem from provided examples

In [ ]:
file = [f for f in files_available if "j301_10.sm" in f][0]
model = parse_file(file)
print(model)

In [ ]:
print(model, "\n", model.mode_details)

The problem includes 32 tasks and 4 ressources.
The precedence relations are stores in ``successors`` attributes : 

In [ ]:
print(model.successors)

Let's look at the precedence graph.

In [ ]:
from discrete_optimization.generic_rcpsp_tools.graph_tools_rcpsp import build_graph_rcpsp_object
import networkx as nx
import matplotlib.pyplot as plt
graph = build_graph_rcpsp_object(model)
graph_nx = graph.graph_nx
dfs = nx.dfs_tree(G=graph_nx, source=1, depth_limit=10)
shortest_path_length = nx.shortest_path_length(dfs, 1)
length_to_nodes = {}
position = {}
for node in sorted(shortest_path_length, key=lambda x: shortest_path_length[x]):
    length = shortest_path_length[node]
    while not(length not in length_to_nodes or len(length_to_nodes[length]) <= 5):
        length += 1
    if length not in length_to_nodes:
        length_to_nodes[length] = []
    length_to_nodes[length] += [node]
    position[node] = (length, len(length_to_nodes[length]))
nx.draw_networkx(graph_nx, pos=position)
plt.show()

# Compute critical path

The critical path in project management is the longest path in the problem that can't be compressed, therefore it is a lower bound on the optimal makespan that is reachable. It represents a path in the precedence graph.
To compute the critical, one can compute a largest path in the precedence constraints. 

In [ ]:
import networkx as nx
for edge in graph_nx.edges():
    graph_nx[edge[0]][edge[1]]["min_duration"] = min([model.mode_details[edge[1]][mode]["duration"]
                                                      for mode in model.mode_details[edge[1]]])
    graph_nx[edge[0]][edge[1]]["minus_min_duration"] = -graph_nx[edge[0]][edge[1]]["min_duration"]
path = nx.astar_path(G=graph_nx,
                     source=model.source_task,
                     target=model.sink_task,
                     heuristic=lambda x, y: -100 if x!=y else 0,
                     weight="minus_min_duration")
# Or alternatively
# path = nx.dag_longest_path(G=graph_nx, weight='min_duration', 
#                            default_weight=1, topo_order=None)
edges = [(n1, n2) for n1, n2 in zip(path[:-1], path[1:])]
duration = sum(graph_nx[n[0]][n[1]]["min_duration"] for n in edges)
print("Duration of critical path : ", duration)

### Plot the critical path : 

In [ ]:
fig, ax = plt.subplots(1)
nx.draw_networkx(graph_nx, pos=position, ax=ax)
nx.draw_networkx_edges(graph_nx, pos=position, edgelist=edges, edge_color="r", ax=ax)

In [ ]:
from discrete_optimization.rcpsp.solver.cpm import CPM, CPMObject

In [ ]:
CPM??

## Other procedure to compute critical path or minimum project duration

The critical path can be computed by a graph procedure described in https://www.youtube.com/watch?v=4oDLMs11Exs. It is a quite simple, forward and backward graph exploration. In the end it provides earliest start date, earliest finish date, and their counterpart (for a optimal schedule) : latest start date, latest finish date.

In [ ]:
solver = CPM(rcpsp_model=model)
critical_path = solver.run_classic_cpm()
edges = [(pi, pi1) for pi, pi1 in zip(critical_path[:-1], critical_path[1:])]
print(solver.map_node[model.sink_task])

The critical path can be identified as nodes where all the values are equals.

In [ ]:
fig, ax = plt.subplots(1)
nx.draw_networkx(graph_nx, pos=position, ax=ax)
nx.draw_networkx_edges(graph_nx, pos=position, edgelist=edges, edge_color="r", ax=ax)

For ressource constrained scheduling problems, this forward/backward pass is not sufficient to compute a schedule, because the ressource capacity constraints are not taken into account. However the *ESD*, *LSD*, *EFD*, *LFD* values can be used in various heuristics to schedule tasks by priority.

## Plotting a solution

In [ ]:
some_solution = model.get_dummy_solution()

In [ ]:
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view, plot_task_gantt
plot_ressource_view(model, some_solution)
plot_task_gantt(model, some_solution)

## SGS : Serial Generation Scheme


![image.png](images/sgs.png)

SGS algorithms is an procedure that aims at building *feasible* schedule from a permutation of task. The task are inserted in the priority order they are in the permutation list, as soon as possible.

## Exercise : 
code the SGS algorithm based on the previous algorithm !

In [ ]:
import numpy as np
def sgs_algorithm(rcpsp_model: RCPSPModel, 
                  permutation_of_task: List[Hashable], predecessors=None):
    # Compute predecessors for each task. 
    if predecessors is None:
        predecessors = {k: set() for k in rcpsp_model.tasks_list}
        for k in rcpsp_model.successors:
            succ = rcpsp_model.successors[k]
            for s in succ:
                predecessors[s].add(k)
    # duration of the tasks.
    duration_task = {k: rcpsp_model.mode_details[k][1]["duration"] for k in rcpsp_model.mode_details}

    # Schedule to fill..
    schedule = {k: {"start_time": None,
                    "end_time": None}
                for k in rcpsp_model.tasks_list}
    
    
    resources_availability = {r: rcpsp_model.get_resource_availability_array(r) 
                              for r in rcpsp_model.resources_list}
    while True:
        # Select task to be scheduled at this round...
        # etc
        
        schedule[??]["start_time"] = ?
        schedule[??]["end_time"] = ?
    return schedule    

If you are blocked, you can retrieve one corrected version of the SGS by decommenting the following cell : 

In [ ]:
# %load correction/nb1_correction.py

## Testing the sgs : 
From the sgs output, it is quite easy to rebuild a RCPSPSolution object and check if it returns a feasible schedule, by calling the ".satisfy()" function.

In [ ]:
tasks_list_permutation = list(model.tasks_list)
import random
random.shuffle(tasks_list_permutation)
schedule = sgs_algorithm(model, tasks_list_permutation)
print(schedule)
solution = RCPSPSolution(problem=model, rcpsp_schedule=schedule)
print(model.satisfy(solution))

Evaluate : 

In [ ]:
model.evaluate(solution)

### Build a permutation based on critical path computation output :
SGS can be seen as a priority based greedy algorithm, the more the task id is on the left side of the permutation, the more it has chance to be scheduled faster. 
We can therefore build heuristic ordering of the task and run SGS on it. One idea it to reuse output of the CPM algorithm to schedule first the task that have the lowest earliest finish date for example, but you can imagine other rules : 

In [ ]:
# list sorted by EFD ?
perm_efd = sorted(model.tasks_list, key=lambda x: solver.map_node[x]._EFD)
sol_efd = sgs_algorithm(model, perm_efd)
solution_efd = RCPSPSolution(problem=model, rcpsp_schedule=sol_efd)
print("Available fields in CPM output : ", solver.map_node[1].__dict__.keys())

perm_esd = sorted(model.tasks_list, key=lambda x: solver.map_node[x]._ESD)
sol_esd = sgs_algorithm(model, perm_esd)
solution_esd = RCPSPSolution(problem=model, rcpsp_schedule=sol_esd)

# Try different methods ?
# What would be your best results ?
print("EFD ", model.evaluate(solution_efd))
print("ESD ", model.evaluate(solution_esd))

In [ ]:
plot_ressource_view(model, solution_efd)

In [ ]:
print("EFD ", model.evaluate(solution_efd))
print("ESD ", model.evaluate(solution_esd))

Can you find other priority rule to get better results ?

## [OPTIONAL] Bonus for those interested : code a Local search or Genetic algorithm using the SGS algorithms and the permutation encoding !

https://en.wikipedia.org/wiki/Simulated_annealing, https://en.wikipedia.org/wiki/Hill_climbing